Dataset available at https://cricsheet.org/downloads/ipl.zip

has one yaml per match held.  Each file contains ball-by-ball data for the corresponding match.

In order to analyse the data properly, it needs to be converted to a CSV format.

In the following set of code snippets, we're creating the content one record in the CSV from the data from one of the yaml files.  Writing to CSV isn't done, but instead prints to the console.

In [ ]:
import yaml

f = open(r'D:\Official\GitHub\Python_Snippets\files\335982.yaml')

match = dict(yaml.load(f))

In [ ]:
def next_over_ball(over, ball):
    if ball < 12:
        ball += 1
    elif ball == 12 and over <= 19:
        over += 1
        ball = 1
    else:
        over = 0
        ball = 1

    return (over, ball)

In [ ]:
for each_inning in match['innings']:
    for inning in each_inning:
        print(inning)
        deliveries = iter(each_inning[inning]['deliveries'])

        over = 0
        ball = 0
        num_wickets = 0
        num_deliveries = 0
        num_runs = 0

        delivery = next(deliveries)

        while over * ball < 12*19:
            # num_deliveries += 1
            over, ball = next_over_ball(over, ball)
            next_ball = f'{over}.{ball}'
            for info in delivery:
                if next_ball != str(info):
                    print(f'{next_ball} not bowled...')
                    if ball == 12:
                        print (f'd{over} = {num_deliveries}, w{over} = {num_wickets}, r{over} = {num_runs}')
                        num_wickets = 0
                        num_deliveries = 0
                        num_runs = 0
                else:
                    num_deliveries += 1
                    print(f"Scored {delivery[info]['runs']['total']} in the {info} ball.")
                    num_runs += delivery[info]['runs']['total']
                    try:
                        info = delivery[info]
                        info['wicket']
                        num_wickets += 1
                    except:
                        pass
                    try:
                        delivery = next(deliveries)  #move to the next delivery
                    except StopIteration:
                        break